<a href="https://colab.research.google.com/github/jmakings/MiXCR-Pipeline/blob/main/MiXCR_pipeline_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This file contains various shell scripts for concatenating paired end reads .fastq files across lanes as well as MiXCR scripts**

### To utilize scripts with command line input (those with `$1` and/or `$2`) you will need to copy the cell into a text editor and create a .sh file to use command line input

# Concatenating lanes in paired end data

In [ ]:
import numpy as np

In [ ]:
%%shell 
#!/bin/bash

# this script for concatenating all lanes for each paired end read together

# final output is two fastq files, one for each paired end read

# input: either fastq or fastq.gz, whichever file type you are trying to concatenate

printf "\n"
echo "Input Type: $1"

FASTQs=`ls *.$1`

SAMPVEC=$( ls $FASTQs | sed 's!.*/!!' | awk '{m=split($0,a,"_L"); for(i=1;i<m;i++)printf a[i]}; {print ""}' | sort | uniq )

for SAMP in $(echo $SAMPVEC)
do
  printf "\n"
  echo $SAMP

  printf "\n"
  echo "Merging R1"
  cat ${SAMP}_L00*_R1_001.$1 > ${SAMP}_ME_R1_001.$1
  
  printf "\n"
  echo "Merging R2"
  cat ${SAMP}_L00*_R2_001.$1 > ${SAMP}_ME_R2_001.$1
done


In [ ]:
%%shell 
#!/bin/bash

# this script for concatenating a specific sample's reads 
# (mostly for testing purposes, don't use for more than a few files)

# final output is two fastq files, one for each paired end read

# input: either fastq or fastq.gz, whichever file type you are trying to concatenate

printf "\n"
echo "Input Type: $1"

printf "\n"
echo "Sample: $2"
SAMP=$2

printf "\n"
echo "Merging R1"
cat ${SAMP}_L00*_R1_001.$1 > ${SAMP}_ME_R1_001.$1

printf "\n"
echo "Merging R2"
cat ${SAMP}_L00*_R2_001.$1 > ${SAMP}_ME_R2_001.$1


# MiXCR run on concatenated FASTQs

In [ ]:
%%shell 
#!/bin/bash

# this script to obtain the merged FASTQs only and run MiXCR on the merged files
# specific for merged files created previously (with ${SAMP}_ME_R1_001)
# run in the same directory that FASTQs are located

# get fastq files
FASTQs=`ls *ME*`

# File Extensions
SAMPVEC=$( ls $FASTQs | sed 's!.*/!!' | awk '{m=split($0,a,"_M"); for(i=1;i<m;i++)printf a[i]}; {print ""}' | sort | uniq )

#Getting the paired samples as input

for SAMP in $( echo $SAMPVEC );
do  
  printf "\n"
  echo "SAMPLE##################"
  echo $SAMP

  MR1="${SAMP}_ME_R1_001.fastq.gz"
  MR2="${SAMP}_ME_R2_001.fastq.gz"

  echo $MR1
  echo $MR2
  NAME="${SAMP:7:-1}"

  echo "Final Sample Name: ${NAME}"
  
## MIXCR RUN

# Creates 4 types of files:
# .vdjca files with binary alignments (many for each sample)
# .clns files with binary clonotype information (one per sample),
# report files that provide overview of alignment, assembly, extension steps
# and finally .tsv with clonotype information (one for each TCR chain type)
  mixcr analyze shotgun -s hsa \
  --starting-material rna --report ${NAME}_report.txt \
  --receptor-type tcr $MR1 $MR2 ${NAME}
done

# Pulling more data from clonotype files
More information on fields to extract found at https://docs.milaboratories.com/mixcr/reference/mixcr-export/#common-fields


This script creates 2 tsv files for each sample: one for TCR B and one for TCR A/D. 

The preset exported values (denoted by `-p full`) include cloneID, cloneCount, cloneFraction, CDR3 nucleotide and amino acid sequence, target qualities (ASCII-encoded PHRED score of each nucleotide), V,D,J,C hits, scores, and alignments, and minimum PHRED score of the sequence. 

Also included is CDR3 length, average PHRED score of the target sequence, and V,D,J,C hits, genes, and hitscores split into seperate columns. This information is already included in the preset export, however splitting these into seperate columns may be useful in downstream analysis.

Also included is columns with nucleotides split for VCDR3, V-D Junction (N1), DCDR3, D-J Junction (N2), and JCDR3. *These were put together to visually look like Adaptive's output.*

Next is the V-J Junction, for clones without a D gene (TCR A/D and a surprising amount of TCR B)




In [ ]:
%%shell 
#!/bin/bash

# get each clone file in a vector
CLNS=`ls *.clns`

for SAMP in $CLNS
do
  BASE=${SAMP%.*}

# TCR B clone export
  mixcr exportClones \
  --chains TRB \
  -p full \
  -lengthOf CDR3 \
  -nFeature VCDR3Part \
  -nFeature VDJunction \
  -nFeature DCDR3Part \
  -nFeature DJJunction \
  -nFeature JCDR3Part \
  -nFeature VJJunction \
  -avrgFeatureQuality CDR3 \
  -vHit -vGene -vHitScore \
  -dHit -dGene -dHitScore \
  -jHit -jGene -jHitScore \
  -cHit -cGene -cHitScore \
  ${SAMP} ${BASE}_TRB_full.tsv

# TCR A/D clone export
  mixcr exportClones \
  --chains TRAD \
  -p full \
  -lengthOf CDR3 \
  -nFeature VCDR3Part \
  -nFeature VDJunction \
  -nFeature DCDR3Part \
  -nFeature DJJunction \
  -nFeature JCDR3Part \
  -nFeature VJJunction \
  -avrgFeatureQuality CDR3 \
  -vHit -vGene -vHitScore \
  -dHit -dGene -dHitScore \
  -jHit -jGene -jHitScore \
  -cHit -cGene -cHitScore \
  ${SAMP} ${BASE}_TRAD_full.tsv
done


# Postanalysis (Not used at this time)

**Update 9/14/22: Many diversity measures found to be incorrect, likely due to bugs in MiXCR software. Immunarach should be used instead**

Most important parts of this for Payel: 

1.   Diversity file (contains many useful diversity measures for each individual sample)
2.   Preprocessing file (contains info on productive clones and how many were filtered out)

https://docs.milaboratories.com/mixcr/reference/mixcr-postanalysis/


In [ ]:
%%shell 
#!/bin/bash

# Next, we will run postanalysis to get diversity measures for each individual sample file.

# Pairwise postanalysis can also be done, however are many combinations (n^2) for
# n samples 

# get clones files 
CLNS=`ls *.clns`
FIRST="TRUE"
TRUE="TRUE"

for SAMP in $CLNS
do
  mixcr postanalysis individual \
  --default-downsampling none \
  --default-weight-function none \
  --only-productive \
  --preproc-tables postanalysis_preproc/${SAMP}.preproc.tsv \
  --tables postanalysis_tables/${SAMP}.tsv \
  --chains TRB $SAMP "${SAMP}.TRB.json"

  mixcr postanalysis individual \
  --default-downsampling none \
  --default-weight-function none \
  --only-productive \
  --preproc-tables postanalysis_preproc/${SAMP}.preproc.tsv \
  --tables postanalysis_tables/${SAMP}.tsv \
  --chains TRAD $SAMP "${SAMP}.TRAD.json"

  if [ "$FIRST" = "$TRUE" ]; then
    FIRST="FALSE"
    cp postanalysis_tables/"${SAMP}.diversity.TRB.tsv" combined_diversity.TRB.tsv
    cp postanalysis_tables/"${SAMP}.diversity.TRAD.tsv" combined_diversity.TRAD.tsv
  else
    cat postanalysis_tables/"${SAMP}.diversity.TRB.tsv" >> combined_diversity.TRB.tsv
    cat postanalysis_tables/"${SAMP}.diversity.TRAD.tsv" >> combined_diversity.TRAD.tsv
  fi
done

mkdir diversity
mv combined_diversity* diversity/

$f_i$ = frequency of clonotype i

Shannon-Weiner Diversity:

the exponent of clonotype frequency distribution entropy
$$ 1 + \sum f_i^2 $$

Normalized Shannon-Wiener Index:

normalized (divided by log of the number of clonotypes) entropy of clonotype frequency distribution
$$ \exp \left(-\sum f_i \log (f_i)\right) $$

Inverse Simpson Index:
$$ 1 / \sum f_i^2 $$

Chao1 estimate: 
Chao lower bound total diversity estimate

Efron-Thisted estimate: 
lower bound total diversity estimate

d50: 
number of clones substituing a half of the sample abundance

# QC (Not available at this time)
QC isn't available in the software yet. But, it could be useful when it becomes available
https://docs.milaboratories.com/mixcr/reference/mixcr-exportQc/

In [ ]:
%%shell 
#!/bin/bash

# examples of how this can be used

# Exports alignment reports, with a pdf showing the rate of alignment 
mixcr exportQc align \
    [--absolute-values] \ 
    sample1.(vdjca|clns|clna)... \
    aligQc.(pdf|eps|svg|png|jpg)  

# Exports anchor point (FR1/2/3/4, CDR1/2/3) coverage, with a pdf 
# showing the fraction of coverage for each read and overlap
mixcr exportQc coverage \
    [--show-boundaries] \
    alignments.vdjca \
    coverage.(pdf|eps|svg|png|jpg)  

# Exports chain usage summary in alignments (.vdjca) or clonotpyes (.clns) files
mixcr exportQc chainUsage \
    [--absolute-values] \
    [--chains <chains>]... \
    [--hide-non-functional] \
    sample1.(vdjca|clns|clna)... \
    chainUsage.(pdf|eps|svg|png|jpg)  